In [5]:
import json
import pandas as pd
from IPython.display import display, Markdown

def read_experiment_data(file_path):
    """读取实验数据文件"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def extract_experiment_info(experiments):
    """提取实验信息"""
    results = []

    for exp in experiments:
        config = exp.get("config", {})
        result = exp.get("result", {})

        # 提取所需的参数
        eval_dataset = config.get("eval_dataset", "N/A")
        edge_cloud_bandwidth = config.get("edge_cloud_bandwidth", 0)
        edge_end_bandwidth = config.get("edge_end_bandwidth", 0)
        eval_mode = result.get("eval_mode", "")
        accuracy = result.get("accuracy", "N/A")  # 提取 accuracy
        target_forward_times = result.get("target_forward_times", 0)
        wall_time = result.get("wall_time", 0)
        communication_time = result.get("communication_time", 0)
        edge_cloud_data_bytes = result.get("edge_cloud_data_bytes", 0)
        throughput = result.get("throughput", 0)
        comm_energy = result.get("comm_energy", 0)

        results.append(
            {
                "数据集": eval_dataset,
                "Edge-Cloud 带宽 (Mbps)": edge_cloud_bandwidth,
                "Edge-End 带宽 (Mbps)": edge_end_bandwidth,
                "方法": eval_mode,
                "准确率": accuracy,  # 添加到结果中
                "目标模型 Forward 次数": target_forward_times,
                "挂钟时间 (s)": round(wall_time, 2),
                "通信时间 (s)": round(communication_time, 2),
                "Edge-Cloud 数据传输量 (bytes)": edge_cloud_data_bytes,
                "Token 输出速率 (tokens/s)": round(throughput, 2),
                "通信能量": comm_energy,
            }
        )

    return results


def generate_markdown_table(data):
    """生成 markdown 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 生成 markdown 表格
    markdown_table = df_sorted.to_markdown(index=False, tablefmt="github")
    return markdown_table


def get_pandas_table(data):
    """生成 pandas DataFrame 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )
    return df_sorted

def generate_grouped_tables(data):
    """按方法分组生成多个表格"""
    df = pd.DataFrame(data)

    # 按方法分组
    grouped = df.groupby("方法")

    all_tables = []

    for method, group_df in grouped:
        # 按Edge-Cloud带宽降序排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 生成表格
        table = group_df_sorted.to_markdown(index=False, tablefmt="github")

        all_tables.append(f"## {method}\n\n{table}\n")

    return "\n".join(all_tables)


def jupyter_display(experiment_data):
    """
    一个专门为 Jupyter Notebook 设计的主函数，
    用于读取、处理数据并以美观的表格形式显示结果。
    """
    # 创建
    df = pd.DataFrame(experiment_data)
    # --- 1. 显示统一排序的表格 ---
    # 使用 display(Markdown(...)) 来渲染标题
    display(Markdown("# 实验结果对比表（按方法和带宽排序）"))

    # 对 DataFrame 进行排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 直接显示 DataFrame 对象，Jupyter 会将其渲染为 HTML 表格
    display(df_sorted)
    # --- 2. 显示按方法分组的表格 ---
    display(Markdown("# 实验结果对比表（按方法分组）"))

    # 按方法进行分组
    grouped = df.groupby("方法")
    # 遍历每个分组并分别显示
    for method, group_df in grouped:
        # 显示分组标题
        display(Markdown(f"## {method}"))

        # 对当前分组的 DataFrame 进行排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 显示排序后的分组 DataFrame
        display(group_df_sorted)


def main():
    # 读取实验数据
    file_path = "experiment_summary_20260115_164915.json"
    experiments = read_experiment_data(file_path)
    experiment_data = extract_experiment_info(experiments)
    unified_table = generate_markdown_table(experiment_data)
    grouped_tables = generate_grouped_tables(experiment_data)

    jupyter_display(experiment_data)

    # 保存到文件
    with open("experiment_results_table.md", "w", encoding="utf-8") as f:
        f.write("# 实验结果对比表（按方法和带宽排序）\n\n")
        f.write(unified_table)
        f.write("\n\n" + "=" * 80 + "\n\n")
        f.write("# 实验结果对比表（按方法分组）\n\n")
        f.write(grouped_tables)

    print("\n表格已保存到 experiment_results_table.md 文件中")


if __name__ == "__main__":
    main()

# 实验结果对比表（按方法和带宽排序）

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
4,eval/eval_mt_bench.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
9,eval/eval_humaneval.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
14,eval/eval_cnndm.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
19,eval/eval_xsum.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
24,eval/eval_gsm8k.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
2,eval/eval_mt_bench.py,34.6,563,dist_spec,1.75000,5328,622.80,36.46,25850858,32.88,18.229139
7,eval/eval_humaneval.py,34.6,563,dist_spec,1.75000,5328,622.80,36.46,25850858,32.88,18.229139
12,eval/eval_cnndm.py,34.6,563,dist_spec,1.75000,5328,622.80,36.46,25850858,32.88,18.229139
17,eval/eval_xsum.py,34.6,563,dist_spec,1.75000,5328,622.80,36.46,25850858,32.88,18.229139
22,eval/eval_gsm8k.py,34.6,563,dist_spec,1.75000,5328,622.80,36.46,25850858,32.88,18.229139


# 实验结果对比表（按方法分组）

## adaptive_tridecoding

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
4,eval/eval_mt_bench.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
9,eval/eval_humaneval.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
14,eval/eval_cnndm.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
19,eval/eval_xsum.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307
24,eval/eval_gsm8k.py,34.6,563,adaptive_tridecoding,1.71875,3685,663.95,3.07,1947736,32.06,1.534307


...

## large

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
0,eval/eval_mt_bench.py,34.6,563,large,1.7875,20480,933.17,0.0,0,21.95,0.0
5,eval/eval_humaneval.py,34.6,563,large,1.7875,20480,933.17,0.0,0,21.95,0.0
10,eval/eval_cnndm.py,34.6,563,large,1.7875,20480,933.17,0.0,0,21.95,0.0
15,eval/eval_xsum.py,34.6,563,large,1.7875,20480,933.17,0.0,0,21.95,0.0
20,eval/eval_gsm8k.py,34.6,563,large,1.7875,20480,933.17,0.0,0,21.95,0.0


## uncertainty_decoding

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
3,eval/eval_mt_bench.py,34.6,563,uncertainty_decoding,1.63125,575,683.4,3.83,27175044,29.97,1.916291
8,eval/eval_humaneval.py,34.6,563,uncertainty_decoding,1.63125,575,683.4,3.83,27175044,29.97,1.916291
13,eval/eval_cnndm.py,34.6,563,uncertainty_decoding,1.63125,575,683.4,3.83,27175044,29.97,1.916291
18,eval/eval_xsum.py,34.6,563,uncertainty_decoding,1.63125,575,683.4,3.83,27175044,29.97,1.916291
23,eval/eval_gsm8k.py,34.6,563,uncertainty_decoding,1.63125,575,683.4,3.83,27175044,29.97,1.916291



表格已保存到 experiment_results_table.md 文件中
